### 栅格数据处理：重采样、裁剪、拼接、光谱指数计算

遥感影像重采样  
gdalwarp, https://gdal.org/programs/gdalwarp.html

In [6]:
width, height = 1000, 1000
path_in = 'data/data-base/region1_l8_20190504_wgs84.tif'
path_out = 'data/data-section-5/rsimg_resam_gdal.tif'


In [7]:
!gdalwarp -co COMPRESS=LZW -ts $width $height -r bilinear $path_in $path_out


Creating output file that is 1000P x 1000L.
Processing data/data-base/region1_l8_20190504_wgs84.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


波段合并  
gdal_merge.py, https://gdal.org/programs/gdal_merge.html


In [12]:
path_out = 'data/data-base/s2_2bands_gdal.tif'
path_band1 = 'data/data-base/S2B_MSIL2A_20240415T033539_N0510_R061_T48RTN_20240415T062333.SAFE/GRANULE/L2A_T48RTN_A037124_20240415T034512/IMG_DATA/R10m/T48RTN_20240415T033539_B02_10m.jp2'
path_band2 = 'data/data-base/S2B_MSIL2A_20240415T033539_N0510_R061_T48RTN_20240415T062333.SAFE/GRANULE/L2A_T48RTN_A037124_20240415T034512/IMG_DATA/R10m/T48RTN_20240415T033539_B03_10m.jp2'


In [13]:
!gdal_merge.py -ot uint16 -separate -o $path_out $path_band1 $path_band2


0...10...20...30...40...50...60...70...80...90...100 - done.


影像裁剪   
gdal_translate, https://gdal.org/programs/gdal_translate.html   
gdalwarp, https://gdal.org/programs/gdalwarp.html  

In [18]:
path_shp = 'data/data-section-5/subset_1.gpkg'
path_in = 'data/data-base/s2_6bands.tif'
path_out = 'data/data-section-5/subset_1_gdal.tif'


In [50]:
## crop to the extent of shapefile.
import geopandas as gdp
shp_gpd = gdp.read_file(path_shp)
left, bottom, right, up =  shp_gpd.bounds['minx'].values, shp_gpd.bounds['miny'].values,  \
                            shp_gpd.bounds['maxx'].values, shp_gpd.bounds['maxy'].values 

extent = '%f  %f %f %f' % (left, up, right, bottom)
# shp_gpd.bounds
print(extent)


265585.394613  2748538.891315 285806.265252 2735642.115553


/var/folders/n0/3pzbdww50rgbs420szyd042c0000gn/T/ipykernel_48539/2420816591.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  extent = '%f  %f %f %f' % (left, up, right, bottom)


In [49]:
# extent: str(ulx) str(uly) str(lrx) str(lry), e.g., extent='72 38 84 34'
!gdal_translate -projwin $extent -co COMPRESS=LZW $path_in $path_out


Input file size is 5490, 5490
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
## mask the region out of the shapefile extent.
!gdalwarp -cutline $path_shp -crop_to_cutline -overwrite $path_input $path_output


影像拼接    
gdal_merge.py, https://gdal.org/programs/gdal_merge.html

In [5]:
path_img1 = 'data/data-section-5/subset_1.tif'
path_img2 = 'data/data-section-5/subset_2_shift.tif'
path_out = 'data/data-section-5/img_mosaic_gdal.tif'


In [6]:
### the last image is on the top layer.
!gdal_merge.py -init 0 -co COMPRESS=LZW -o $path_out $path_img1 $path_img2 


0...10...20...30...40...50...60...70...80...90...100 - done.
